In [20]:
#https://medium.com/@samlupton/spotipy-get-features-from-your-favourite-songs-in-python-6d71f0172df0

#spotipy has a wealth of other functionality, including getting all songs from a playlist or extracting the popularity of a track, so do explore away!

In [14]:
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
from tqdm import tqdm_notebook
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '8e6e962237b74490aca58fd858c5a54f'
client_secret = '06fd25cc7b164f7e88927938c5c1a589'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

def artist_tracks(artists):
    
    '''
    Takes a list of artist names, iterates through their Spotify albums, checks for 
    duplicate albums, then appends all the tracks in those albums to a list of lists
    '''
    
    # Each list in this list will be a track and its features
    tracks = []
    
    
    
    for artist in tqdm_notebook(artists):
        
        # Get the artist URI (a unique ID)
        artist_uri = sp.search(artist)['tracks']['items'][0]['artists'][0]['uri']

        # Spotify has a lot of duplicate albums, but we'll cross-reference them with this list to avoid extra loops
        album_checker = []
        
        # The starting point of our loop of albums for those artists with more than 50
        n = 0
        
        # Note the album_type = 'album'. This discounts singles, compilations and collaborations
        while len(sp.artist_albums(artist_uri, album_type = 'album', limit=50, offset = n)['items']) > 0:
            
            # Avoid overloading Spotify with requests by assigning the list of album dictionaries to a variable
            dict_list = sp.artist_albums(artist_uri, album_type = 'album', limit=50, offset = n)['items']
            
            for i, album in tqdm_notebook(enumerate(dict_list)):

                # Add the featured artists for the album in question to the checklist
                check_this_album = [j['name'] for j in dict_list[i]['artists']]
                # And the album name
                check_this_album.append(dict_list[i]['name'])
                # And its date
                check_this_album.append(dict_list[i]['release_date'])

                # Only continue looping if that album isn't in the checklist
                if check_this_album not in album_checker:
                    
                    # Add this album to the checker
                    album_checker.append(check_this_album)
                    # For every song on the album, get its descriptors and features in a list and add to the tracklist
                    tracks.extend([[artist, album['name'], album['uri'], song['name'],

                      album['release_date']] + list(sp.audio_features(song['uri'])[0].values()) 
                                   for song in sp.album_tracks(album['uri'])['items']])
            
            # Go through the next 50 albums (otherwise we'll get an infinite while loop)
            n += 50

    return tracks

In [15]:
artist_tracks(['The Beatles'])

<ipython-input-14-eface1997f7e>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for artist in tqdm_notebook(artists):


  0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-14-eface1997f7e>:40: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, album in tqdm_notebook(enumerate(dict_list)):


0it [00:00, ?it/s]

[['The Beatles',
  'Let It Be (Super Deluxe)',
  'spotify:album:1BdxbYp1FaNejpDgtDo25V',
  'Two Of Us - 2021 Mix',
  '2021-10-15',
  0.692,
  0.756,
  7,
  -9.677,
  1,
  0.0402,
  0.0356,
  8.8e-05,
  0.219,
  0.826,
  109.111,
  'audio_features',
  '2CEujT1Tuv3jIIjUyUvupg',
  'spotify:track:2CEujT1Tuv3jIIjUyUvupg',
  'https://api.spotify.com/v1/tracks/2CEujT1Tuv3jIIjUyUvupg',
  'https://api.spotify.com/v1/audio-analysis/2CEujT1Tuv3jIIjUyUvupg',
  216480,
  4],
 ['The Beatles',
  'Let It Be (Super Deluxe)',
  'spotify:album:1BdxbYp1FaNejpDgtDo25V',
  'Dig A Pony - 2021 Mix',
  '2021-10-15',
  0.461,
  0.5,
  11,
  -8.023,
  0,
  0.0478,
  0.183,
  9.94e-06,
  0.209,
  0.393,
  118.377,
  'audio_features',
  '5pbQlOzqzTacF4wbzmWEeA',
  'spotify:track:5pbQlOzqzTacF4wbzmWEeA',
  'https://api.spotify.com/v1/tracks/5pbQlOzqzTacF4wbzmWEeA',
  'https://api.spotify.com/v1/audio-analysis/5pbQlOzqzTacF4wbzmWEeA',
  234760,
  3],
 ['The Beatles',
  'Let It Be (Super Deluxe)',
  'spotify:album:1B

In [16]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '8e6e962237b74490aca58fd858c5a54f'
client_secret = '06fd25cc7b164f7e88927938c5c1a589'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

def df_tracks(tracklist):
    
    '''
    Takes the output of artist_tracks (i.e. a list of lists),
    puts it in a dataframe and formats it.
    '''

    df = pd.DataFrame(tracklist, columns=['artist',
     'album_name',
     'album_uri',
     'track',
     'release_date'] + list(sp.audio_features('7tr2za8SQg2CI8EDgrdtNl')[0].keys()))

    df.rename(columns={'uri':'song_uri'}, inplace=True)

    df.drop_duplicates(subset=['artist', 'track', 'release_date'], inplace=True)

    # Reorder the cols to have identifiers first, auditory features last
    cols = ['artist', 'album_name', 'album_uri', 'track', 'release_date', 'id', 'song_uri', 'track_href',
     'analysis_url', 'type', 'danceability', 'energy', 'key',  'loudness', 'mode', 'speechiness',
     'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

    df = df[cols]
    
    return df

In [18]:
tracklist = artist_tracks(['The Beatles'])

<ipython-input-14-eface1997f7e>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for artist in tqdm_notebook(artists):


  0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-14-eface1997f7e>:40: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, album in tqdm_notebook(enumerate(dict_list)):


0it [00:00, ?it/s]

In [19]:
df_tracks(tracklist)

,artist,album_name,album_uri,track,release_date,id,song_uri,track_href,analysis_url,type,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,The Beatles,Let It Be (Super Deluxe),spotify:album:1BdxbYp1FaNejpDgtDo25V,Two Of Us - 2021 Mix,2021-10-15,2CEujT1Tuv3jIIjUyUvupg,spotify:track:2CEujT1Tuv3jIIjUyUvupg,https://api.spotify.com/v1/tracks/2CEujT1Tuv3j...,https://api.spotify.com/v1/audio-analysis/2CEu...,audio_features,...,-9.677,1,0.0402,0.0356,0.000088,0.2190,0.826,109.111,216480,4
1,The Beatles,Let It Be (Super Deluxe),spotify:album:1BdxbYp1FaNejpDgtDo25V,Dig A Pony - 2021 Mix,2021-10-15,5pbQlOzqzTacF4wbzmWEeA,spotify:track:5pbQlOzqzTacF4wbzmWEeA,https://api.spotify.com/v1/tracks/5pbQlOzqzTac...,https://api.spotify.com/v1/audio-analysis/5pbQ...,audio_features,...,-8.023,0,0.0478,0.1830,0.000010,0.2090,0.393,118.377,234760,3
2,The Beatles,Let It Be (Super Deluxe),spotify:album:1BdxbYp1FaNejpDgtDo25V,Across The Universe - 2021 Mix,2021-10-15,47q5q2VZ0y264CNDdUAjZY,spotify:track:47q5q2VZ0y264CNDdUAjZY,https://api.spotify.com/v1/tracks/47q5q2VZ0y26...,https://api.spotify.com/v1/audio-analysis/47q5...,audio_features,...,-9.189,1,0.0270,0.5520,0.000023,0.0779,0.752,76.148,228093,4
3,The Beatles,Let It Be (Super Deluxe),spotify:album:1BdxbYp1FaNejpDgtDo25V,I Me Mine - 2021 Mix,2021-10-15,5sFSXKwsf1KvKpZngwVtlI,spotify:track:5sFSXKwsf1KvKpZngwVtlI,https://api.spotify.com/v1/tracks/5sFSXKwsf1Kv...,https://api.spotify.com/v1/audio-analysis/5sFS...,audio_features,...,-8.140,0,0.0415,0.1400,0.000000,0.1180,0.575,128.470,145053,4
4,The Beatles,Let It Be (Super Deluxe),spotify:album:1BdxbYp1FaNejpDgtDo25V,Dig It - 2021 Mix,2021-10-15,211vMasdcN1n4Xy4ci76Eq,spotify:track:211vMasdcN1n4Xy4ci76Eq,https://api.spotify.com/v1/tracks/211vMasdcN1n...,https://api.spotify.com/v1/audio-analysis/211v...,audio_features,...,-9.868,1,0.0742,0.4520,0.004860,0.2730,0.795,155.067,50653,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,The Beatles,Please Please Me (Remastered),spotify:album:3KzAvEXcqJKBF97HrXwlgf,Baby It's You - Remastered 2009,1963-03-22,2VmB1rF9FtfKUmFHDVnq8Q,spotify:track:2VmB1rF9FtfKUmFHDVnq8Q,https://api.spotify.com/v1/tracks/2VmB1rF9FtfK...,https://api.spotify.com/v1/audio-analysis/2VmB...,audio_features,...,-12.211,0,0.0345,0.7780,0.000000,0.0926,0.879,112.421,160520,4
580,The Beatles,Please Please Me (Remastered),spotify:album:3KzAvEXcqJKBF97HrXwlgf,Do You Want To Know A Secret - Remastered 2009,1963-03-22,7Aobt67JnaF7qN8jCCKvHq,spotify:track:7Aobt67JnaF7qN8jCCKvHq,https://api.spotify.com/v1/tracks/7Aobt67JnaF7...,https://api.spotify.com/v1/audio-analysis/7Aob...,audio_features,...,-12.414,1,0.0368,0.6080,0.000000,0.3800,0.609,124.451,117013,4
581,The Beatles,Please Please Me (Remastered),spotify:album:3KzAvEXcqJKBF97HrXwlgf,A Taste Of Honey - Remastered 2009,1963-03-22,7fh53ta3vAOGJMQ4i5tCHe,spotify:track:7fh53ta3vAOGJMQ4i5tCHe,https://api.spotify.com/v1/tracks/7fh53ta3vAOG...,https://api.spotify.com/v1/audio-analysis/7fh5...,audio_features,...,-11.416,0,0.0327,0.6980,0.000000,0.1040,0.412,101.408,123480,3
582,The Beatles,Please Please Me (Remastered),spotify:album:3KzAvEXcqJKBF97HrXwlgf,There's A Place - Remastered 2009,1963-03-22,4dessGxnKXmTbHPhVgqODq,spotify:track:4dessGxnKXmTbHPhVgqODq,https://api.spotify.com/v1/tracks/4dessGxnKXmT...,https://api.spotify.com/v1/audio-analysis/4des...,audio_features,...,-10.009,1,0.0292,0.6290,0.000004,0.1720,0.927,140.928,110493,4
